In [6]:
import os
import sys
from antlr4 import *
from ExprLexer import ExprLexer
from ExprParser import ExprParser
from ExprListener import ExprListener
from ExprVisitor import ExprVisitor

print("Import Successful!")

Import Successful!


# listener

In [7]:
class MyExprListener(ExprListener):
    def enterProg(self, ctx:ExprParser.ProgContext):
        print("Entering program")

    def exitProg(self, ctx:ExprParser.ProgContext):
        print("Exiting program")

    def enterStat(self, ctx:ExprParser.StatContext):
        print(f"Entering statement: {ctx.getText()}")

    def exitStat(self, ctx:ExprParser.StatContext):
        print(f"Exiting statement: {ctx.getText()}")

    def enterExpr(self, ctx:ExprParser.ExprContext):
        print(f"Entering expression: {ctx.getText()}")

    def exitExpr(self, ctx:ExprParser.ExprContext):
        print(f"Exiting expression: {ctx.getText()}")


In [11]:
input_path = "examples/expr.txt"

input_stream = FileStream(input_path)
lexer = ExprLexer(input_stream)
stream = CommonTokenStream(lexer)
parser = ExprParser(stream)
tree = parser.prog()

listener = MyExprListener()
walker = ParseTreeWalker()
walker.walk(listener, tree)

ANTLR runtime and generated code versions disagree: 4.10!=4.13.1
ANTLR runtime and generated code versions disagree: 4.10!=4.13.1
Entering program
Entering statement: 193

Entering expression: 193
Exiting expression: 193
Exiting statement: 193

Entering statement: a=5

Entering expression: 5
Exiting expression: 5
Exiting statement: a=5

Entering statement: b=6

Entering expression: 6
Exiting expression: 6
Exiting statement: b=6

Entering statement: a+b*2

Entering expression: a+b*2
Entering expression: a
Exiting expression: a
Entering expression: b*2
Entering expression: b
Exiting expression: b
Entering expression: 2
Exiting expression: 2
Exiting expression: b*2
Exiting expression: a+b*2
Exiting statement: a+b*2

Entering statement: (1+2)*3<missing NEWLINE>
Entering expression: (1+2)*3
Entering expression: (1+2)
Entering expression: 1+2
Entering expression: 1
Exiting expression: 1
Entering expression: 2
Exiting expression: 2
Exiting expression: 1+2
Exiting expression: (1+2)
Entering ex

line 5:7 missing NEWLINE at '<EOF>'


# visitor

In [27]:
class MyExprVisitor(ExprVisitor):
    def visitProg(self, ctx:ExprParser.ProgContext):
        print("Visiting program")
        return self.visitChildren(ctx)

    def visitStat(self, ctx:ExprParser.StatContext):
        print(f"Visiting statement: {ctx.getText()}")
        return self.visitChildren(ctx)

    def visitExpr(self, ctx:ExprParser.ExprContext):
        if ctx.getChildCount() == 1:  # INT or ID
            return ctx.getText()
        elif ctx.getChildCount() == 3:
            left = self.visit(ctx.getChild(0))
            op = ctx.getChild(1).getText()
            right = self.visit(ctx.getChild(2))
            return f"({left} {op} {right})"
        elif ctx.getChildCount() == 2:  # Parens (expr)
            return self.visit(ctx.getChild(1))

    def visitTerminal(self, node):
        return node.getText()

In [28]:
visitor = MyExprVisitor()
result = visitor.visit(tree)
print(f"Result: {result}")

Visiting program
Visiting statement: 193

Visiting statement: a=5

Visiting statement: b=6

Visiting statement: a+b*2

Visiting statement: (1+2)*3<missing NEWLINE>
Result: None


In [29]:
class MyExprVisitor(ExprVisitor):
    def visitProg(self, ctx:ExprParser.ProgContext):
        """
        Visits each statement in the program, collects the results, and returns them as a list.
        """
        results = []
        for stat in ctx.stat():
            result = self.visit(stat)
            if result is not None:
                results.append(result)
        return results

    def visitStat(self, ctx:ExprParser.StatContext):
        if ctx.expr():
            return self.visit(ctx.expr())
        elif ctx.ID():
            var_name = ctx.ID().getText()
            value = self.visit(ctx.expr())
            return f"{var_name} = {value}"
        else:
            return None

    def visitExpr(self, ctx:ExprParser.ExprContext):
        if ctx.getChildCount() == 1:  # INT or ID
            return ctx.getText()
        elif ctx.getChildCount() == 3:
            left = self.visit(ctx.getChild(0))
            op = ctx.getChild(1).getText()
            right = self.visit(ctx.getChild(2))
            return f"({left} {op} {right})"
        elif ctx.getChildCount() == 2:  # Parens (expr)
            return self.visit(ctx.getChild(1))


In [30]:
visitor = MyExprVisitor()
result = visitor.visit(tree)
print(f"Result: {result}")

Result: ['193', '5', '6', '(a + (b * 2))', '((None 1+2 None) * 3)']
